In [ ]:
"""
@authors: faurand, chardes, ehagensieker
"""
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np 

#load the mnist dataset
(train_ds, test_ds), ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info = True)


Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to ~/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:

import math
import datetime

# in a notebook, load the tensorboard extension, not needed for scripts
%load_ext tensorboard

In [ ]:
def prepare_mnist_data(task,mnist):
  
  #flatten the images into one dimensional vector
  mnist = mnist.map(lambda img, target: (tf.reshape(img, (-1,)), target))
  #convert data from uint8 to float32
  mnist = mnist.map(lambda img, target: (tf.cast(img, tf.float32), target))
  #normalization to have input of range [-1,1]
  mnist = mnist.map(lambda img, target: ((img/128.)-1., target))
  #create one-hot vector targets
  #mnist = mnist.map(lambda img, target: (img, tf.one_hot(target, depth=10)))
  mnist = tf.data.Dataset.zip((mnist.shuffle(2000), mnist.shuffle(2000)))
  #mnist = mnist.map(lambda zip(pic1,pic2): (pic1[0],pic2[0],pic1[1],pic2[1]))
  mnist = calc(task,mnist)
  #mnist = mnist.map(lambda img, target: (img, target ))
  #cache this progress in memory - improve performance
  mnist = mnist.cache()
  #shuffle, batch, prefetch
  mnist = mnist.shuffle(1000)
  mnist = mnist.batch(32)
  mnist = mnist.prefetch(20)

  #return preprocessed dataset
  return mnist

#apply the preprocessing to both data sets
# train_1 = prepare_mnist_data(1,train_ds)
# test_1 = prepare_mnist_data(1,test_ds)

# train_2 = prepare_mnist_data(2,train_ds)
# test_2 = prepare_mnist_data(2,test_ds)

In [ ]:
def calc(task,mnist):
  if task == 1:
    mnist = mnist.map(lambda pic1,pic2: (pic1[0],pic2[0],tf.cast((pic1[1] + pic2[1] >= 5), tf.int32)))
    return mnist
  if task == 2:
    mnist = mnist.map(lambda pic1,pic2: (pic1[0],pic2[0], pic1[1]-pic2[1]))
    mnist = mnist.map(lambda img1,img2,target: (img1,img2,tf.one_hot(target, depth=19)))
    return mnist
  


In [ ]:
for i in train_2.take(1):
  print("pic1:", i[0], "\npic2: ", i[1], "\ntarget: ",i[2])

In [ ]:
"""
@authors: faurand, chardes, ehagensieker
"""
from tensorflow.keras.layers import Dense

class MyModel(tf.keras.Model):
    
    def __init__(self,task=1):
        #initialize the model with n fully connected layer
        #inherits from tf.keras.Model
        super(MyModel, self).__init__()

        self.dense1 = tf.keras.layers.Dense(256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(256, activation=tf.nn.relu)
        #self.dense3 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
        #self.dense1 = tf.keras.layers.Dense(10, activation=tf.nn.relu)
        if task==1:
          self.metrics_list = [tf.keras.metrics.Mean(name = 'loss'),tf.keras.metrics.BinaryAccuracy(name = "acc")]
          self.loss_function = tf.keras.losses.BinaryCrossentropy()
          self.out_layer = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
        elif task==2:
          self.metrics_list = [tf.keras.metrics.Mean(name = 'loss'),tf.keras.metrics.CategoricalAccuracy(name = "acc")]
          self.loss_function = tf.keras.losses.CategoricalCrossentropy()
          self.out_layer = tf.keras.layers.Dense(19, activation=tf.nn.softmax)
        else: print("Exception")

        #self.learning_rate = 0.001
        self.optimizer = tf.keras.optimizers.Adam()

    @tf.function 
    def call(self, images):
      """
      how to forward the images through the layer
      """
      img1, img2 = images 

      img1_x = self.dense1(img1)
      img1_x = self.dense2(img1_x)

      img2_x = self.dense1(img2)
      img2_x = self.dense2(img2_x)

      combined = tf.concat([img1_x, img2_x], axis = 1)
      out = self.out_layer(combined)

      return out

    @property
    def metrics(self):
      return self.metrics_list


    def reset_metric(self):
      for metric in self.metrics: 
        metric.reset_states()

    @tf.function 
    def train_step(self, data):
      img1, img2, t = data

      with tf.GradientTape() as tape:
        output = self((img1, img2), training = True)
        loss = self.loss_function(t, output)

      gradients = tape.gradient(loss, self.trainable_variables)
      self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
      self.metrics[0].update_state(loss)
      # # for all metrics except loss, update states (accuracy etc.)
      for metric in self.metrics[1:]:
          metric.update_state(t,output)
      
      return {metric.name: metric.result() for metric in self.metrics}

    @tf.function
    def test_step(self, data):
      img1,img2, t = data

      prediction = self((img1,img2), training = False)
      loss = self.loss_function(t, prediction)

      self.metrics[0].update_state(loss)
      for metric in self.metrics[1:]:
          metric.update_state(t, prediction)
          
      return {metric.name: metric.result() for metric in self.metrics}

    

In [ ]:
# Define where to save the log
config_name= "config_name"
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

train_log_path = f"logs/{config_name}/{current_time}/train"
val_log_path = f"logs/{config_name}/{current_time}/val"

# log writer for training metrics
train_summary_writer = tf.summary.create_file_writer(train_log_path)

# log writer for validation metrics
val_summary_writer = tf.summary.create_file_writer(val_log_path)

In [ ]:
import pprint
import tqdm

def training_loop(model, train_ds, val_ds, epochs, train_summary_writer, val_summary_writer):
    for epoch in range(epochs):
        print(f"Epoch {epoch}:")
        
        # Training:
        
        for data in tqdm.tqdm(train_ds, position=0, leave=True):
            metrics = model.train_step(data)
            
            # logging the validation metrics to the log file which is used by tensorboard
            with train_summary_writer.as_default():
                for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)

        # print the metrics
        print([f"{key}: {value.numpy()}" for (key, value) in metrics.items()])

        # reset all metrics (requires a reset_metrics method in the model)
        model.reset_metrics()    
        
        # Validation:
        for data in val_ds:
            metrics = model.test_step(data)
        
            # logging the validation metrics to the log file which is used by tensorboard
            with val_summary_writer.as_default():
                for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)
                    
        print([f"val_{key}: {value.numpy()}" for (key, value) in metrics.items()])

        # reset all metrics
        model.reset_metrics()
        print("\n")

In [ ]:
def run(task):
  if task == 1:
    train = prepare_mnist_data(1,train_ds)
    test = prepare_mnist_data(1,test_ds)
  if task == 2:
    train = prepare_mnist_data(2,train_ds)
    test = prepare_mnist_data(2,test_ds)
    
  train_dataset = train.take(1000)
  test_dataset = test.take(100)

    ### Hyperparameters
  num_epochs = 2
  

  # Initialize the model.
  model = MyModel(task)

  training_loop(model,train_dataset,test_dataset,num_epochs,train_summary_writer, val_summary_writer)


In [ ]:
run(2)

Epoch 0:


100%|██████████| 1000/1000 [00:20<00:00, 48.61it/s]


['loss: 1.0051546096801758', 'acc: 0.47359374165534973']
['val_loss: 0.8592202663421631', 'val_acc: 0.5640624761581421']


Epoch 1:


100%|██████████| 1000/1000 [00:20<00:00, 49.53it/s]


['loss: 0.8422827124595642', 'acc: 0.5996249914169312']
['val_loss: 0.7816745638847351', 'val_acc: 0.6637499928474426']


